In [5]:
"""
    1 * 1卷积层可以用于 融合通道
    
    总结的技巧：
        假设输出的高宽不变，通道数一般不动
        假设输出的高宽减半，通道数一般翻一倍
"""
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K): # 交叉相关
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))


# 验证互相关运算的输出 
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                    [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])

K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)


tensor([[ 56.,  72.],
        [104., 120.]])

In [6]:
# 计算多个通道的输出的互相关函数
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0) # 在0的这个维度上堆起来
K.shape

torch.Size([3, 2, 2, 2])

In [7]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [8]:
# 1*1 卷积
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape # 通道数、高、宽
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w)) # 把高、宽拉成一条
    K = K.reshape((c_o, c_i)) # 也拉成二维
    Y = torch.matmul(K, X) # 矩阵乘法
    return Y.reshape((c_o, h, w)) # 拉回三维

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1)) # 输出2， 输入3，kelnel：1*1

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6 # 证明了1*1的卷积等价于全连接

In [ ]:
# 将高度和宽度的步幅设置为2
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
